### In the next cell, the libraries and the file needed for the project are imported

In [10]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import requests
import lxml.html as lh

def __iter__(self): return 0

body = client_7a207381b1af4d15b126270709dfd5b8.get_object(Bucket='courseracapstoneproject-donotdelete-pr-x1dwa7gkvsd5xa',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_lat_long = pd.read_csv(body)
df_lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Url is saved, as well as the table content

In [11]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

### Separate the first line of the table and save it in a list to be the table header

In [12]:
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


### Add the following data to the list associating it with the data previously added

In [13]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]

    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 

        col[i][1].append(data)

        i+=1

### Turn list into a dictionary and then into a dataframe

In [14]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

### Remove rows in de dataframe where Borough is Not assigned

In [15]:
df = df[df.Borough != 'Not assigned']

### Rename df column 'Neighbourhood\n' to 'Neighborhood'

In [16]:
df = df.rename(columns={'Neighbourhood\n': 'Neighborhood'})
df.columns

Index(['Postcode', 'Borough', 'Neighborhood'], dtype='object')

### Remove /n from the Neighborhood column values

In [17]:
df['Neighborhood'] = df['Neighborhood'].str[:-1]

### Replace Neighbourhood column values that are equal to 'Not assigned' to the Borough column value of the same row

In [18]:
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df['Borough']

### Use concat to do an inner join with the dataframe from the file and the dataframe from the Url

In [20]:
result = pd.concat([df, df_lat_long], axis=1, join='inner')

### Drop the second Postcode column

In [26]:
result = result.drop(result.columns[3],axis=1)

In [28]:
result

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476
5,M6A,North York,Lawrence Heights,43.744734,-79.239476
6,M6A,North York,Lawrence Manor,43.727929,-79.262029
7,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577
9,M9A,Queen's Park,Queen's Park,43.692657,-79.264848
10,M1B,Scarborough,Rouge,43.757410,-79.273304
11,M1B,Scarborough,Malvern,43.750072,-79.295849
13,M3B,North York,Don Mills North,43.781638,-79.304302
